In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json
import csv

In [47]:
# URL of the CUNY Fall 2021 academic calendar site
url = 'https://www.ccny.cuny.edu/registrar/fall'

In [48]:
# Send a GET request to the URL
response = requests.get(url)

In [49]:
response

<Response [200]>

In [50]:
page_text = response.text

In [51]:
page_text[:200]

'<!DOCTYPE html>\n<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http:'

In [52]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

In [53]:
content =soup.article

In [54]:
content

<article about="/registrar/fall" role="article" typeof="schema:WebPage">
<div class="layout-content--inner__wrapper white-bg">
<div>
<div class="breadcrumbs">
<div class="breadcrumbs__inner">
<span class="breadcrumbs__carat">&gt;</span>
<a class="breadcrumbs__breadcrumb" href="/">Home</a>
<span class="breadcrumbs__slash">/</span>
<a class="breadcrumbs__breadcrumb" href="/registrar">Registrar</a>
<span class="breadcrumbs__slash">/</span>
<a class="breadcrumbs__breadcrumb active" href="">Fall 2021 Academic Calendar</a>
</div>
</div>
</div>
<div class="layout-content--inner">
<div class="content--container--wrapper">
<div class="content--container__full">
<div class="body-paragraph paragraph">
<p> </p>
<h4>Dates are subject to change<br/>
December 07, 2021</h4>
<p> </p>
<table align="center" border="3" cellpadding="1" cellspacing="1" style="width:1272px"><thead><tr><th class="text-align-left" scope="col" style="width: 305px;">DATES</th>
<th class="text-align-left" scope="col" style="width

In [55]:
# Initialize lists to store extracted data
links_list = []

In [56]:
#Loop through all the links in the article
for link in content.find_all("a"):
  try: #try to get url,date,days of week and text
    url = link.get("href")
    date = link.date.get("DATE")
    days_of_week = link.days_of_week.get("DAYS")
    text = link.span.text
    links_list.append({'url': url, 'date': date, 'days_of_week' : days_of_week, 'text': text})
  except AttributeError: #if rows are missing or anythin
    pass #skip it don't blow up

In [66]:

#save as json file
with open('links.json', 'w', encoding ='utf-8') as links_file:
  json.dump(links_list, links_file, ensure_ascii = False )


In [67]:
#save it as csv
with open('links.csv', 'w', newline ='') as csv_out:
  csv_writer = csv.writer(csv_out)
  #create the header row
  csv_writer.writerow(['url','date','days_of_week','text'])
  for row in links_list:
    csv_writer.writerow([str(row['url']),str(row['date']),str(row['days_of_week']),str(row['text'])])


In [79]:
df = pd.DataFrame(links_list)
file_path = "links_list.json"
df.to_json(file_path, orient="records")
